In [9]:

import GPSPreprocess as gpsp
import pytest
import geopandas as gpd
import pandas as pd
import os
import sys
import re
import datetime

sample_gps_file_path = os.getcwd().split(
"PyERT-BLACK")[0] + 'PyERT-BLACK/quarto-example/data/sample-gps'

sample_gps = f'{sample_gps_file_path}/sample-gps-1.csv'
Data = pd.read_csv(sample_gps)
DataLength = len(Data)
GPSData = gpsp.GPSPreprocess(data=Data)

ProcessedData = GPSData.getData()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(ProcessedData.head(1000))


        
Processed = ProcessedData.reset_index()
stage_points = {}
pattern = '(\d+)\/(\d+)\/(\d+)\s(\d+):(\d+)'
prev_time = 0
episode_len = 0
temp_episode_len = 0
start_index = 0
ep_id= 0
valid_stop = False
last_car_index = -1

for index,row in ProcessedData.iterrows():
    #print(index)
    speed = float(row['Speed_kmh'])
    #print(speed)
    
    
    if (speed <= 10.008):
        new_mode = 'walk/stop' 
        if (speed > 0.36): 
            valid_stop = False
    elif (speed > 10.008):
        new_mode = 'car'

    pattern = '(\d+)\/(\d+)\/(\d+)\s(\d\d):(\d\d)'
    match = re.search(pattern, row['LocalTime'])
    cur_time = datetime.datetime(year = int(match.groups()[2]),month = int(match.groups()[1]), day = int(match.groups()[0]), hour = int(match.groups()[3]), minute = int(match.groups()[4]))
    if index == 0:
        cur_mode = new_mode
        prev_time = cur_time
    time_difference = (cur_time - prev_time).total_seconds()
    prev_time = cur_time
    episode_len += time_difference
    
    
    if (time_difference > 120): #GAP
        #print(f'GAP time_difference: {episode_len} cur_mode: {cur_mode}')
        ep_id+=1
        stage_points[ep_id] = {}
        stage_points[ep_id]['start'] = start_index
        stage_points[ep_id]['end'] = index
        stage_points[ep_id]['mode'] = cur_mode
        episode_len = 0
        start_index = index + 1
        cur_mode = new_mode
    else:
        if (new_mode != cur_mode):
            if (new_mode == 'walk/stop') and cur_mode == 'car': #currently, no invalid car episodes
                if (last_car_index < 0):
                    last_car_index = index - 1
                    temp_episode_len = 0

                temp_episode_len += time_difference
                #print('temp_episode_len: ' + str(temp_episode_len))
                if ( temp_episode_len > 60 ):
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = last_car_index
                    stage_points[ep_id]['mode'] = 'car'
                    start_index = last_car_index + 1
                    last_car_index = -1
                    episode_len = temp_episode_len
                    cur_mode = new_mode
            elif new_mode == "car" and (cur_mode == 'walk/stop') :  
                if ((episode_len > 60 and valid_stop)): #valid stop
                    #print(f'valid episode ep_len: {episode_len} cur_mode: {cur_mode} new_mode: {new_mode}')
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = index - 1
                    stage_points[ep_id]['mode'] = 'stop'
                elif ((episode_len > 60 and not valid_stop)): #invalid stop, valid walk
                    #print(f'valid episode ep_len: {episode_len} cur_mode: {cur_mode} new_mode: {new_mode}')
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = index - 1
                    stage_points[ep_id]['mode'] = 'walk'
                else :
                    #print(f'invalid episode')
                    if (ep_id == 0):
                        stage_points[ep_id] = {}
                        stage_points[ep_id]['start'] = start_index
                        stage_points[ep_id]['end'] = index - 1
                        stage_points[ep_id]['mode'] = 'invalid'
                    if (ep_id != 0):
                        stage_points[ep_id]['end'] = index - 1
                    
                start_index = index
                episode_len = 0
                cur_mode = new_mode
                valid_stop = True
        else:
            if temp_episode_len > 0:
                last_car_index = -1
                temp_episode_len = 0

serial_ids = []
record_ids = []
start_times = []
modes = []
geometry = []

for key in stage_points:
    print(f"key: {key} start: {stage_points[key]['start']} end: {stage_points[key]['end']} mode: {stage_points[key]['mode']}")
    serial_ids.append((ProcessedData.iloc[stage_points[key]['start']]).SerialID)
    record_ids.append((ProcessedData.iloc[stage_points[key]['start']]).RecordID)
    start_times.append((ProcessedData.iloc[stage_points[key]['start']]).LocalTime)
    modes.append(stage_points[key]['mode'])
    geometry.append((ProcessedData.iloc[stage_points[key]['start']]).geometry)

data = { 'EPISODEID' : serial_ids, 
        'RECORDID' : record_ids, 
        'TIMESTART' : start_times, 
        'MODES' : modes,
        'geometry': geometry}
gdf = gpd.GeoDataFrame(data)
            

# ProcessedDataLength = len(ProcessedData)

#segmentation into singele mode stages

#three types of potential mode transfer points: end of walk, start of walk, end of gap

#speed and acceleration threshold for walking: 2.78 m/s, 0.1 m/s^2

#each walking stage enclosed by SOW (or SOG) and EOW (or EOG)

#walking stage minimal duration: 60s

#other modes: 120s

#mode assignment 



key: 0 start: 0 end: 4 mode: invalid
key: 1 start: 5 end: 320 mode: car
key: 2 start: 321 end: 575 mode: car
key: 3 start: 576 end: 910 mode: car
key: 4 start: 911 end: 984 mode: car
key: 5 start: 985 end: 1020 mode: walk
key: 6 start: 1021 end: 3010 mode: car
key: 7 start: 3011 end: 3259 mode: walk
key: 8 start: 3260 end: 4004 mode: car
key: 9 start: 4005 end: 4321 mode: car
key: 10 start: 4322 end: 7428 mode: car
key: 11 start: 7429 end: 7462 mode: walk
key: 12 start: 7463 end: 8091 mode: car
key: 13 start: 8092 end: 8346 mode: car
